In [1]:
import numpy as np
import yfinance as yf
from coskweness_cokurtosis import coskewness, cokurtosis
from portfolio_hubo_qaoa import HigherOrderPortfolioQAOA
import random
random.seed(4)

n_stocks = 3
stocks = ["AAPL", "GOOGL", "MSFT", "AMZN", "META", "TSLA", "WMT", "CAT", "KO", "IBM", "INTC", "CSCO", "ORCL", "QCOM", 
          "NVDA", "ADBE", "PYPL", "CRM", "ACN", "TXN", "AVGO", "INTU", "AMAT", "MU"]
stocks = random.sample(stocks, n_stocks)

data = yf.download(stocks, start="2021-01-01", end="2024-01-01")
prices_now = data["Close"].iloc[-1]
print(prices_now)
returns = data["Close"].pct_change().dropna()
stocks = returns.columns

numpy_returns = returns.to_numpy()
expected_returns = numpy_returns.mean(axis=0)*252
print(returns.mean())
print(expected_returns)
covariance_matrix = np.cov(numpy_returns, rowvar=False)*252
coskewness_tensor = coskewness(numpy_returns)#*(252**2)
cokurtosis_tensor = cokurtosis(numpy_returns)#*(252**3)

# Budget in dollars
# We compute the maximum number of shares we can buy for each stock
# At most n_qubits for each stock
n_qubits = 3
cheapest_stock = prices_now.min()
budget = n_qubits*cheapest_stock
print(budget)

risk_aversion = 3

portfolio_hubo = HigherOrderPortfolioQAOA(stocks=stocks,
                                          prices_now=prices_now,
                                          expected_returns=expected_returns, 
                                          covariance_matrix=covariance_matrix,
                                          budget=budget,
                                          coskewness_tensor=coskewness_tensor, 
                                          cokurtosis_tensor=cokurtosis_tensor,
                                          log_encoding = True, 
                                          layers = 5,
                                          risk_aversion = risk_aversion)

smallest_eigenvalues, smallest_bitstrings, first_excited_energy, optimized_portfolio, second_optimized_portfolio = portfolio_hubo.solve_exactly()

[*********************100%***********************]  3 of 3 completed

Ticker
AMZN    151.940002
CAT     289.939758
IBM     156.799332
Name: 2023-12-29 00:00:00, dtype: float64
Ticker
AMZN    0.000214
CAT     0.000900
IBM     0.000712
dtype: float64
[0.05393377 0.22692002 0.17942043]
455.82000732421875
Number of qubits for asset AMZN: 2
Number of qubits for asset CAT: 1
Number of qubits for asset IBM: 2
Number of qubits per asset:  {'AMZN': 2, 'CAT': 1, 'IBM': 2}
Constructing cost hubo with integer variables
Optimized Weights (considering variance, skewness and kurtosis):
AMZN: 17.06%
CAT: 53.43%
IBM: 29.51%
Left over budget:  9.080917329977126
Optimized Discrete Allocation:
CAT: 1
IBM: 1
Maximized utility from classical higher moments:  32.04574801808301
Adding budget constraints to the cost function -> constructing full hubo problem
Replacing integer variables with binary variables
Simplifying the binary cost function
Converting binary cost function to Ising Hamiltonian
Constructing QAOA circuits


In [2]:
print(smallest_eigenvalues)
print(smallest_bitstrings)
print(first_excited_energy)
print("Best: ", optimized_portfolio)
print("Second best: ", second_optimized_portfolio)
print("objective value: ", portfolio_hubo.get_objective_value(stocks, optimized_portfolio[0]))

[np.float64(-6658157.019803236)]
[[1, 1, 0, 0, 0]]
-6657450.086190378
Best:  [{'AMZN': 3, 'CAT': 0, 'IBM': 0}]
Second best:  [{'AMZN': 2, 'CAT': 0, 'IBM': 1}]
objective value:  48.262343550432576


In [ ]:
two_most_probable_states, smallest_eigenvalue, params, total_steps, states_probs, optimized_portfolios = portfolio_hubo.solve_with_iterative_QAOA(max_layers=5)

Trying with 1 layers


In [ ]:
print(two_most_probable_states)
print(smallest_eigenvalue)
print(params)
print(total_steps)
print(states_probs)
print(optimized_portfolios)

['1010', '1001']
-236.73045377737114
[[ 0.05488863  1.69755863  1.28786653  1.61175958  1.33918021]
 [-0.54515017 -0.4550527   2.15697143  0.21207511  0.06597024]]
1000
[tensor(0.1658505, requires_grad=True), tensor(0.40148811, requires_grad=True)]
[{'CAT': 1, 'IBM': 1}, {'CAT': 1, 'IBM': 2}]


In [ ]:
from portfolio_higher_moments_classical import HigherMomentPortfolioOptimizer


hef = HigherMomentPortfolioOptimizer(expected_returns=expected_returns, 
                                     covariance_matrix=covariance_matrix, 
                                     coskewness=coskewness_tensor, 
                                     cokurtosis=cokurtosis_tensor, 
                                     risk_aversion=risk_aversion)

weights = hef.optimize_portfolio_with_higher_moments()
print("Optimized weights:")
for asset, weight in zip(stocks, weights):
    print(f"{asset}: {weight:.2%}")

Optimized weights:
CAT: 64.36%
IBM: 35.64%
